# 작업 내용
- 선정기간 동안 5개 이상의 article을 읽은 독자에 한해서 분석 진행
- 독자 1명이 읽은 글의 전체 태그를 말뭉치로 만듦
- 말뭉치를 형태소로 분석
    - Doc2vec 적용하여 유사한 독자 서칭
    - 독자 간 cosine 유사도 분석

In [1]:
from konlpy.tag import Kkma
import matplotlib.pyplot as plt
from tqdm import tqdm
from pandas import Panel
from sklearn.feature_extraction.text import TfidfVectorizer
from pandas import Panel
tqdm.pandas()
from sklearn.metrics.pairwise import cosine_similarity

C:\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  after removing the cwd from sys.path.
C:\anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  
C:\anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
from itertools import chain

In [ ]:
from Brunch_data import *

In [ ]:
meta = pd.read_json('C:/Users/eunice/dss_workspace/MLproject/data/metadata.json', lines=True)
read_file_lst = glob.glob('C:/Users/eunice/dss_workspace/MLproject/data/read\*')
maga = pd.read_json('C:/Users/eunice/dss_workspace/MLproject/data/magazine.json', lines=True)
us = pd.read_json('C:/Users/eunice/dss_workspace/MLproject/data/users.json', lines=True)

In [ ]:
a = Data(meta, read_file_lst, maga, us)
me_df, r_df, mag_df, us_df = a.data_preprocessing()
# 2/15 - 3/31 train 데이터
train = a.train_data()

In [ ]:
# article_id, keyword_list
key = a.train_keyword_list()

In [ ]:
# readers별 article list와 articlle count가 있는 df :ra
ra_list_df = Read_article_outline_remove(train)
ra = ra_list_df.read_article_list()
# up = ra_list_df.upper_fence_remove()
# 5 이상 upper fence 밑으로 정리
# t3 = article_count_division_3(up)

In [ ]:
upper_fence_df = ra[(ra['article_id_count']>=9) & (ra['article_id_count']<=64)]

In [ ]:
upper_fence_df

# 독자-독자 유사도 구하기
- 독자별 주로 읽은 글의 키워드 정리하기
- 1. train2 데이터와 read데이터의 독자 정보 merge
- 2. train2 데이터에서 독자 id기준 groupby/ keyword list sum
- 3. 키워드 중복 제거
- 4. 키워드 형태소 분석

In [ ]:
upper_fence_df.drop(['article_list', 'article_id_count'], axis=1, inplace=True)

## 데이터 만들기

In [ ]:
# 1. train 데이터와 upper_fence_df데이터의 독자 정보 merge (설정 기간동안 작성된 article에 대해서만 병합)
train2 = train.merge(upper_fence_df, how='inner', on='readers_id')
train2

In [ ]:
# 2. 독자 아이디별 태그리스트 합치기
train_key_sum = train2.groupby('readers_id')['keyword_list'].agg(sum)
train_key_sum

In [ ]:
train_key_sum = train_key_sum.reset_index()

In [ ]:
len(train_key_sum)

## 키워드가 있는 작가만 남기기

In [ ]:
# 3. 키워드가 있는 아이디만 남기기
train_key_sum = train_key_sum[train_key_sum['keyword_list'].apply(lambda x: len(x))!=0]
train_key_sum

## 키워드 중복 제거하기

In [ ]:
# 4. 중복제거 명령
tqdm.pandas()
train_key_sum['keyword_list'] = train_key_sum['keyword_list'].progress_apply(lambda x: list(set(x)))

## 키워드 형태소 분석

In [ ]:
kkma = Kkma()

In [ ]:
def morphs_analysis(data):
    ls = []
    for x in range(len(data)):
        ls.append(kkma.morphs(data[x]))
    return list(chain.from_iterable(ls))

In [ ]:
train_key_sum['morphs_list'] = train_key_sum['keyword_list'].progress_apply(lambda x: morphs_analysis(x))

In [ ]:
train_key_sum

In [ ]:
# 중복제거 전 형태소분석 결과 저장하기
# train_key_sum.to_csv('keyword_list_mporphs_duplicates.csv', encoding='utf-8')

In [ ]:
# 형태소 분석 완료 (중복제거 버전)
train_key_sum.to_csv('kewordlist_morphs.csv', encoding='utf-8')

## CountVectorizer 구성: 형태소 벡터화

- *파일로 읽어올 경우! 꼭 replace가 들어간 apply를 실행해주세요* -

In [2]:
train_key_sum = pd.read_csv('kewordlist_morphs.csv', encoding='utf-8')

In [3]:
train_key_sum['morphs_list'] = train_key_sum['morphs_list'].apply(lambda x: str(x).replace('[', '').replace(']','').replace("'", ''))

### countvectorizer instanciation, fit_transform

In [4]:
# keyword_list 유사도 측정 (countvectorizer을 이용해서 빈도수로 벡터화)
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환
tqdm.pandas()
train_key_sum["keyword_list_literal"] = train_key_sum['morphs_list'].progress_apply(lambda x: ("").join(x))
train_key_sum["keyword_list_literal"].dropna(inplace=True)
# 이를 적용해 개별 요소를 공백 문자로 구분하는 문자열로 변환해 별도의 칼럼인 keyword_list_literal 칼럼으로 저장

count_vect = CountVectorizer(min_df=0, ngram_range=(1,3)) 
# min_df : 최소 빈도수(최소 노출 빈도수 정해주는 것), ngram_range: 토큰화 단어갯수
reader_keyword_mat = count_vect.fit_transform(train_key_sum["keyword_list_literal"])
print(reader_keyword_mat.shape)
# 643104 레코드와 1088182개의 개별 단어 피처로 구성된 피처 벡터 행렬이 만들어짐

100%|██████████| 12022/12022 [00:00<00:00, 47485.74it/s]


(12022, 536260)


In [6]:
print(reader_keyword_mat)

  (0, 268392)	1
  (0, 24497)	1
  (0, 253604)	1
  (0, 343155)	1
  (0, 268401)	1
  (0, 26218)	1
  (0, 254102)	1
  (0, 268402)	1
  (0, 26221)	1
  (1, 465682)	1
  (1, 456095)	1
  (1, 161704)	1
  (1, 347161)	1
  (1, 285561)	1
  (1, 17590)	1
  (1, 336754)	1
  (1, 448138)	1
  (1, 288328)	1
  (1, 259879)	1
  (1, 244493)	1
  (1, 184808)	1
  (1, 382908)	1
  (1, 322304)	1
  (1, 374256)	1
  (1, 295584)	1
  :	:
  (12021, 399358)	1
  (12021, 425058)	1
  (12021, 290880)	1
  (12021, 471200)	1
  (12021, 227383)	1
  (12021, 471201)	1
  (12021, 330276)	1
  (12021, 254087)	1
  (12021, 193649)	1
  (12021, 473917)	1
  (12021, 19067)	1
  (12021, 515580)	1
  (12021, 303742)	1
  (12021, 515582)	1
  (12021, 397052)	1
  (12021, 265859)	1
  (12021, 473918)	1
  (12021, 368576)	1
  (12021, 290884)	1
  (12021, 398175)	1
  (12021, 330280)	1
  (12021, 274572)	1
  (12021, 405179)	1
  (12021, 95834)	1
  (12021, 344988)	1


## vector 간 cosine 유사도 구하기

In [5]:
cos_reader_keyword_mat = cosine_similarity(reader_keyword_mat.toarray(), reader_keyword_mat.toarray())
print(cos_reader_keyword_mat.shape)
print(cos_reader_keyword_mat[:2])

MemoryError: Unable to allocate 48.0 GiB for an array with shape (12022, 536260) and data type int64

In [ ]:
pd.DataFrame(cos_reader_keyword_mat)

In [ ]:
reader_keyword_sim_sorted_ind = cos_reader_keyword_mat.argsort()[:,::-1]
print(reader_keyword_sim_sorted_ind[:1])

In [ ]:
def find_sim_article(df, sorted_ind, readers_name, top_n=10):
    reader_list = df[df['readers_id']==readers_name]
    
    article_index = reader_list.index.values
    similar_indexes = sorted_ind[article_index, :(top_n)]
    
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    

    return df.iloc[similar_indexes]

In [ ]:
train_key_sum

In [ ]:
train_key_sum[train_key_sum['readers_id']=='#fffe67ecc0056dd26ae00511957c5a2b']

## cosine 유사도 기준 추천

In [ ]:
similar_article = find_sim_article(train_key_sum, reader_keyword_sim_sorted_ind, '#fffe67ecc0056dd26ae00511957c5a2b', 10)
similar_article[['readers_id', 'keyword_list']]

# 작가-작가 유사도 구하기

## 작가별 키워드 말뭉치 구하기

In [ ]:
# 파일 read하기
author_key_sum = train.groupby('author_id')['keyword_list'].agg('sum')
author_key_sum = author_key_sum.reset_index()
author_key_sum

In [ ]:
len(author_key_sum)

In [ ]:
# 작가별 주로 사용한 글의 키워드
author_key_sum = author_key_sum[author_key_sum['keyword_list'].apply(lambda x: len(x))!=0]
author_key_sum

In [ ]:
# 4. 중복제거 명령
tqdm.pandas()
author_key_sum['k_list'] = author_key_sum['keyword_list'].progress_apply(lambda x: list(set(x)))

In [ ]:
author_key_sum.reset_index(inplace=True)

## 키워드 형태소 분석

In [ ]:
author_key_sum['morphs_list'] = author_key_sum['k_list'].progress_apply(lambda x: morphs_analysis(x))

In [ ]:
# 형태소 분석 결과 저장
author_key_sum.to_csv('author_keyword_list_unique.csv', encoding='utf-8')

## CountVectorizer 구성: 형태소 벡터화

In [ ]:
author_key_sum['morphs_list']

### countvectorizer instanciation, fit_transform

In [ ]:
# keyword_list 유사도 측정 (countvectorizer을 이용해서 빈도수로 벡터화)
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환
author_key_sum["keyword_list_literal"] = author_key_sum['morphs_list'].progress_apply(lambda x: (" ").join(x))
author_key_sum["keyword_list_literal"].dropna(inplace=True)
# 이를 적용해 개별 요소를 공백 문자로 구분하는 문자열로 변환해 별도의 칼럼인 keyword_list_literal 칼럼으로 저장

count_vect = CountVectorizer(min_df=0, ngram_range=(1,3)) 
# min_df : 최소 빈도수(최소 노출 빈도수 정해주는 것), ngram_range: 토큰화 단어갯수
author_keyword_mat = count_vect.fit_transform(author_key_sum["keyword_list_literal"])
print(author_keyword_mat.shape)
# 643104 레코드와 1088182개의 개별 단어 피처로 구성된 피처 벡터 행렬이 만들어짐

In [ ]:
print(author_keyword_mat.toarray())

## vector 간 cosine 유사도 구하기

In [ ]:
cos_author_keyword_mat = cosine_similarity(author_keyword_mat.toarray(), author_keyword_mat.toarray())
print(cos_author_keyword_mat.shape)
print(cos_author_keyword_mat[:2])

In [ ]:
pd.DataFrame(cos_author_keyword_mat)

In [ ]:
author_keyword_sim_sorted_ind = cos_author_keyword_mat.argsort()[:,::-1]
print(author_keyword_sim_sorted_ind[:1])

In [ ]:
def find_sim_author(df, sorted_ind, author_name, top_n=10):
    author_list = df[df['author_id']==author_name]
    
    article_index = author_list.index.values
    similar_indexes = sorted_ind[article_index, :(top_n)]
    
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)

    return df.iloc[similar_indexes]

In [ ]:
author_key_sum

In [ ]:
# testdata에만 있음
author_key_sum[author_key_sum['author_id']== '@13july']['keyword_list']

## cosine 유사도 기준 추천

In [ ]:
author_similar_article = find_sim_author(author_key_sum, author_keyword_sim_sorted_ind, '@13july', 50)
author_similar_article[['author_id','keyword_list']]

# 독자-작가 간 유사도 구하기

In [ ]:
print(author_keyword_mat[0])

In [ ]:
reader_keyword_mat[0].to_array()

In [ ]:
cosine_similarity(reader_keyword_mat[0], author_keyword_mat[0])

In [ ]:
reader_author_keyword_mat = cosine_similarity(author_keyword_mat, reader_keyword_mat)
print(reader_author_keyword_mat.shape)
print(reader_author_keyword_mat[:2])

In [ ]:
pd.DataFrame(reader_author_keyword_mat)

In [ ]:
reader_author_keyword_sim_sorted_ind = reader_author_keyword_mat.argsort()[:,::-1]
print(reader_author_keyword_mat_sorted_ind[:1])

In [ ]:
def find_sim_reader_author(df, sorted_ind, readers_name, top_n=10):
    reader_list = df[df['author_id']==readers_name]
    
    article_index = reader_list.index.values
    similar_indexes = sorted_ind[article_index, :(top_n)]
    
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    

    return df.iloc[similar_indexes]

In [ ]:
# testdata에만 있음
author_keyword_sum[author_keyword_sum['author_id']== '@13july']

In [ ]:
keyword_sim[27964]

## cosine 유사도 기준 추천

In [ ]:
author_similar_article = find_sim_author(author_keyword_sum, author_keyword_sim_sorted_ind, '#d39d7f99b55209e6ecd651481bda9a66', 10)
author_similar_article[['author_id', 'keyword_list']]